**Multi-task and Multi-Modal Learning**

In [ ]:
# Install Autokeras and keras tuner

!pip install autokeras
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc3


     |████████████████████████████████| 143kB 11.0MB/s 
     |████████████████████████████████| 849kB 20.0MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 174kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 49.4MB/s 
     |████████████████████████████████| 358kB 43.0MB/s 
     |████████████████████████████████| 36.7MB 78kB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=95133e59ead50c1942ce666a66b55227ff8da3886727ced56b303ecaea570d0a
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo
  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc3) to /tmp/pip-req-build-jgda_epk
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-jgda_epk
  Running command git checkout -q 172be42f1310cdbad48941494f84d65367de4f11
  Created wheel for ker

In [ ]:
# Import required libraries
import numpy as np
import autokeras as ak

In [ ]:
# Data generation

num_Instances = 150
imageData = np.random.rand(num_Instances, 32, 32, 3).astype(np.float32)
structuredData = np.random.rand(num_Instances, 20).astype(np.float32)


In [ ]:
# Generate targets for regression
regressionTarget = np.random.rand(num_Instances, 1).astype(np.float32)
# Generate  labels of five classes for classification
classificationTarget = np.random.randint(5, size=num_Instances)


In [ ]:
# Build the model using AutoModel

model = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=[
        ak.RegressionHead(metrics=['mae']),
        ak.ClassificationHead(loss='categorical_crossentropy', metrics=['accuracy'])
    ],
    overwrite=True,
    max_trials=2)



In [ ]:
# Train and validate the model
model.fit(
    [imageData, structuredData],
    [regressionTarget, classificationTarget],
    validation_split=0.15,
    epochs=4)


Trial 2 Complete [00h 00m 15s]
val_loss: 1.8149256706237793

Best val_loss So Far: 1.8149256706237793
Total elapsed time: 00h 00m 30s
INFO:tensorflow:Oracle triggered exit
Epoch 1/4
5/5 [==============================] - 0s 72ms/step - loss: 14.1901 - regression_head_1_loss: 10.7715 - classification_head_1_loss: 3.4186 - regression_head_1_mae: 2.6160 - classification_head_1_accuracy: 0.2600
Epoch 2/4
5/5 [==============================] - 0s 33ms/step - loss: 20.2691 - regression_head_1_loss: 16.8160 - classification_head_1_loss: 3.4531 - regression_head_1_mae: 3.0611 - classification_head_1_accuracy: 0.2467
Epoch 3/4
5/5 [==============================] - 0s 34ms/step - loss: 22.2506 - regression_head_1_loss: 18.8216 - classification_head_1_loss: 3.4290 - regression_head_1_mae: 3.3077 - classification_head_1_accuracy: 0.2333
Epoch 4/4
5/5 [==============================] - 0s 33ms/step - loss: 16.7330 - regression_head_1_loss: 13.6586 - classification_head_1_loss: 3.0744 - regression_

## Customized Search Space
You can customize your search space.
The following figure shows the search space we want to define.

<div class="mermaid">
graph LR
    id1(ImageInput) --> id2(Normalization)
    id2 --> id3(Image Augmentation)
    id3 --> id4(Convolutional)
    id3 --> id5(ResNet V2)
    id4 --> id6(Merge)
    id5 --> id6
    id7(StructuredDataInput) --> id8(CategoricalToNumerical)
    id8 --> id9(DenseBlock)
    id6 --> id10(Merge)
    id9 --> id10
    id10 --> id11(Classification Head)
    id10 --> id12(Regression Head)
</div>


In [ ]:
# Customize the search space

input_node_1 = ak.ImageInput()
output_node = ak.Normalization()(input_node1)
output_node = ak.ImageAugmentation()(output_node)
output_node_1 = ak.ConvBlock()(output_node)
output_node_2 = ak.ResNetBlock(version='v2')(output_node)
output_node_1 = ak.Merge()([output_node_1, output_node_2])

input_node_2 = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node_2)
output_node_2 = ak.DenseBlock()(output_node)

output_node = ak.Merge()([output_node1, output_node_2])
output_node_1 = ak.ClassificationHead()(output_node)
output_node_2 = ak.RegressionHead()(output_node)

auto_model = ak.AutoModel(
    inputs=[input_node1, input_node_2], 
    outputs=[output_node1, output_node_2],
    overwrite=True,
    max_trials=2)

image_Data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
structured_Data = np.random.rand(num_instances, 20).astype(np.float32)
regression_Target = np.random.rand(num_instances, 1).astype(np.float32)
classification_Target = np.random.randint(5, size=num_instances)

auto_model.fit(
    [image_Data, structured_Data],
    [classification_Target, regression_Target],
    batch_size=32,
    epochs=3)


Trial 2 Complete [00h 00m 33s]
val_loss: 1.5399234294891357

Best val_loss So Far: 1.5399234294891357
Total elapsed time: 00h 01m 08s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4/4 [==============================] - 5s 1s/step - loss: 3.5350 - classification_head_1_loss: 2.0213 - regression_head_1_loss: 1.5138 - classification_head_1_accuracy: 0.1300 - regression_head_1_mean_squared_error: 1.5138
Epoch 2/3
4/4 [==============================] - 5s 1s/step - loss: 3.9944 - classification_head_1_loss: 1.8507 - regression_head_1_loss: 2.1437 - classification_head_1_accuracy: 0.3200 - regression_head_1_mean_squared_error: 2.1437
Epoch 3/3
4/4 [==============================] - 5s 1s/step - loss: 3.3795 - classification_head_1_loss: 1.8187 - regression_head_1_loss: 1.5608 - classification_head_1_accuracy: 0.2800 - regression_head_1_mean_squared_error: 1.5608
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
